In [1]:
# Make sure autoreload is properly set up for auto reload the module if changes
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from dotenv import load_dotenv, find_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)


_ = load_dotenv(find_dotenv())

# print(os.environ.get('OPENAI_API_KEY'))
print(parent_dir)

/home/ct-admin/Documents/Langgraph/00_AudioProject/02_Call_Center_AI_Agent


In [5]:
from src.Agents.graph_test_call_center import *
# Test individual steps
# test_individual_step("negotiation", "I can't afford this")

# # Test full workflow
# test_full_workflow("cooperative")

# Run compliance verification
# run_architecture_tests()

# # Interactive testing
interactive_test_mode()


INTERACTIVE TEST MODE
Available commands:
1. test_step <step_name> <message>
2. test_workflow <personality>
3. test_router <message> <current_step>
4. list_agents
5. list_personalities
6. quit


Unknown command. Type 'quit' to exit.

Exiting interactive test mode.


In [13]:
# Router Testing Code
"""
Comprehensive router testing for the optimized call center agent
"""
import logging
from typing import Dict, Any, List, Tuple
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Simple client data for testing
client_data = {
    "user_id": "83906",
    "profile": {
        "client_info": {
            "client_full_name": "John Smith",
            "first_name": "John", 
            "title": "Mr",
            "id_number": "8001010001088",
            "email_address": "john.smith@example.com"
        },
        "user_name": "jsmith123",
        "vehicles": [{"registration": "ABC123GP", "make": "Toyota", "model": "Camry", "color": "White"}]
    },
    "account_aging": {"x0": "0.00", "x30": "199.00", "xbalance": "199.00"},
    "account_overview": {"account_status": "Overdue"},
    "subscription": {"subscription_amount": "199.00"},
    "payment_history": [],
    "failed_payments": [],
    "banking_details": {}
}

def create_router_llm():
    """Create router LLM for testing."""
    # model = ChatOllama(model="qwen2.5:14b-instruct", temperature=0, num_ctx=4096)
    model = ChatOllama(model="qwen2.5:7b-instruct-q5_K_M", temperature=0, num_ctx=4096)
    return model

def get_optimized_router_prompt(state):
        """Generate highly optimized step-aware router classification prompt."""
        current_step = state.get("current_step", "unknown")
        
        # Get last client message
        messages = state.get("messages", [])
        last_client_message = ""
        last_ai_message = ""
        
        for msg in reversed(messages):
            if isinstance(msg, dict):
                if msg.get("type") == "human" and not last_client_message:
                    last_client_message = msg.get("content", "")
                elif msg.get("type") == "ai" and not last_ai_message:
                    last_ai_message = msg.get("content", "")
            elif hasattr(msg, 'type'):
                if msg.type == "human" and not last_client_message:
                    last_client_message = msg.content
                elif msg.type == "ai" and not last_ai_message:
                    last_ai_message = msg.content

        # Pre-analyze message for keyword patterns
        msg_lower = last_client_message.lower()
        
        # Determine likely classification based on patterns
        has_escalation_words = any(word in msg_lower for word in [
            "supervisor", "manager", "complaint", "harassment", "legal", "lawyer"
        ])
        
        has_cancellation_words = any(word in msg_lower for word in [
            "cancel", "terminate", "stop service", "end service", "disconnect"
        ])
        
        has_question_words = any(pattern in msg_lower for pattern in [
            "how does", "what happens", "why wasn", "what's included", "how do you"
        ])
        
        has_agreement_words = any(word in msg_lower for word in [
            "okay", "yes", "fine", "alright", "let's do", "sure"
        ])
        
        has_objection_words = any(pattern in msg_lower for pattern in [
            "can't afford", "don't have", "no money", "won't pay", "refuse", "wrong", "dispute"
        ])

        return f"""<role>Debt Collection Call Router</role>

    <message_to_classify>"{last_client_message}"</message_to_classify>

    <context>
    Current Step: {current_step}
    AI Said: "{last_ai_message}"
    </context>

    <classification_rules>
    1. ESCALATION: Contains "supervisor", "manager", "complaint", "harassment", "legal"
    Examples: "I want a supervisor", "This is harassment", "Filing a complaint"

    2. CANCELLATION: Contains "cancel", "terminate", "stop service", "disconnect"  
    Examples: "Cancel my account", "Stop all services", "Terminate this"

    3. QUERY_UNRELATED: Questions about service features/technical issues unrelated to payment
    Examples: "How does Cartrack work?", "What happens if stolen?", "Why wasn't payment taken?"

    4. STEP_RELATED: Directly answers current step's purpose
    Examples: Name confirmations, providing ID, payment discussions, verification responses

    5. AGREEMENT: Clear acceptance of AI's request
    Examples: "Okay let's arrange payment", "Yes I understand", "Fine I'll pay"

    6. OBJECTION: Refusal or resistance to current step  
    Examples: "Can't afford", "Don't have money", "That's wrong", "Won't pay"
    </classification_rules>

    <keyword_analysis>
    Escalation keywords detected: {has_escalation_words}
    Cancellation keywords detected: {has_cancellation_words}  
    Question keywords detected: {has_question_words}
    Agreement keywords detected: {has_agreement_words}
    Objection keywords detected: {has_objection_words}
    </keyword_analysis>

    <step_context>
    For {current_step}:
    - Identity confirmations (name, ID) = STEP_RELATED
    - Payment amount questions = STEP_RELATED
    - Service feature questions = QUERY_UNRELATED
    - Financial refusals = OBJECTION
    - Positive responses to requests = AGREEMENT
    </step_context>

    <critical_examples>
    "I want to speak to a supervisor" → ESCALATION (supervisor keyword)
    "Stop all services immediately" → CANCELLATION (stop + services keywords)  
    "How does Cartrack work?" → QUERY_UNRELATED (service question)
    "Yes, this is John Smith" → STEP_RELATED (identity confirmation)
    "I can't afford R199" → OBJECTION (financial refusal)
    "Okay, let's arrange payment" → AGREEMENT (accepting request)
    "Can I pay half now?" → STEP_RELATED (payment discussion)
    "Why wasn't my payment taken?" → QUERY_UNRELATED (technical question)
    </critical_examples>

    <instructions>
    1. Check for escalation keywords FIRST
    2. Check for cancellation keywords SECOND  
    3. Distinguish questions (QUERY_UNRELATED) from responses (STEP_RELATED)
    4. Financial refusals are always OBJECTION
    5. Clear acceptance is AGREEMENT
    6. Direct responses to current step are STEP_RELATED
    </instructions>

    <output_format>
    Respond with EXACTLY ONE WORD: ESCALATION, CANCELLATION, STEP_RELATED, QUERY_UNRELATED, AGREEMENT, or OBJECTION
    </output_format>"""

def parse_router_decision(llm_response: str) -> str:
    """Parse LLM response and validate routing decision."""
    
    # Extract classification from LLM response
    response_text = llm_response.content if hasattr(llm_response, 'content') else str(llm_response)
    response_upper = response_text.strip().upper()
    
    # Valid classifications
    valid_classifications = [
        "ESCALATION", "CANCELLATION", "STEP_RELATED", 
        "QUERY_UNRELATED", "AGREEMENT", "OBJECTION"
    ]
    
    # Find matching classification
    classification = None
    for valid in valid_classifications:
        if valid in response_upper:
            classification = valid
            break
    
    # Default to STEP_RELATED if unclear
    if not classification:
        classification = "STEP_RELATED"
    
    return classification

def test_router_classification(test_message: str, current_step: str = "negotiation", 
                             ai_context: str = "") -> str:
    """Test the router classification with a specific message."""
    
    print(f"\n🔍 TESTING ROUTER CLASSIFICATION")
    print(f"=" * 50)
    print(f"Current Step: {current_step}")
    print(f"Test Message: '{test_message}'")
    if ai_context:
        print(f"AI Context: '{ai_context}'")
    
    try:
        router_llm = create_router_llm()
        
        # Create test state
        messages = []
        if ai_context:
            messages.append({"type": "ai", "content": ai_context})
        messages.append({"type": "human", "content": test_message})
        
        test_state = {
            "current_step": current_step,
            "messages": messages,
            "client_name": client_data.get('profile', {}).get('client_info', {}).get('client_full_name', 'Client')
        }
        
        # Generate router prompt
        prompt_content = get_optimized_router_prompt(test_state)
        prompt = [SystemMessage(content=prompt_content)]
        
        # Get classification
        response = router_llm.invoke(prompt)
        classification = parse_router_decision(response)
        
        print(f"🎯 Router Classification: {classification}")
        print(f"🤖 Raw Response: {response.content}")
        
        return classification
        
    except Exception as e:
        print(f"❌ Error testing router classification: {e}")
        return "ERROR"

def run_comprehensive_router_tests():
    """Run comprehensive router tests across different scenarios."""
    
    print(f"\n{'='*60}")
    print("COMPREHENSIVE ROUTER TESTING")
    print(f"{'='*60}")
    
    # Test scenarios: (message, current_step, expected_classification, ai_context)
    test_scenarios = [
        # ESCALATION tests
        ("I want to speak to a supervisor", "negotiation", "ESCALATION", ""),
        ("Get me your manager now", "promise_to_pay", "ESCALATION", ""),
        ("I'm filing a complaint", "reason_for_call", "ESCALATION", ""),
        ("This is harassment, I want a supervisor", "negotiation", "ESCALATION", ""),
        
        # CANCELLATION tests
        ("Cancel my account", "promise_to_pay", "CANCELLATION", ""),
        ("I want to terminate this service", "negotiation", "CANCELLATION", ""),
        ("Stop all services immediately", "reason_for_call", "CANCELLATION", ""),
        
        # QUERY_UNRELATED tests
        ("How does Cartrack work?", "negotiation", "QUERY_UNRELATED", "Your account is overdue by R199.00"),
        ("What happens if my car is stolen?", "promise_to_pay", "QUERY_UNRELATED", "Can we debit R199 today?"),
        ("Why wasn't my payment taken?", "reason_for_call", "QUERY_UNRELATED", ""),
        ("What's included in the service?", "details_verification", "QUERY_UNRELATED", ""),
        
        # STEP_RELATED tests
        ("Yes, this is John Smith", "name_verification", "STEP_RELATED", "Are you John Smith?"),
        ("My ID number is 8001010001088", "details_verification", "STEP_RELATED", "Please provide your ID number"),
        ("I can't afford R199 right now", "negotiation", "STEP_RELATED", "Your account is overdue by R199"),
        ("Can I pay half now?", "promise_to_pay", "STEP_RELATED", "Can we debit R199 today?"),
        
        # AGREEMENT tests
        ("Yes, I understand", "negotiation", "AGREEMENT", "Services will stop without payment"),
        ("Okay, let's arrange payment", "promise_to_pay", "AGREEMENT", "Can we debit R199 today?"),
        ("Fine, I'll pay", "negotiation", "AGREEMENT", ""),
        
        # OBJECTION tests
        ("I don't have money right now", "promise_to_pay", "OBJECTION", "Can we debit R199 today?"),
        ("That amount is wrong", "reason_for_call", "OBJECTION", "You owe R199"),
        ("I already paid this", "negotiation", "OBJECTION", ""),
        ("I won't pay until you fix the service", "promise_to_pay", "OBJECTION", ""),
    ]
    
    results = []
    correct_predictions = 0
    
    for i, (message, step, expected, ai_context) in enumerate(test_scenarios, 1):
        print(f"\n📋 Test {i}/{len(test_scenarios)}")
        
        actual = test_router_classification(message, step, ai_context)
        
        is_correct = actual == expected
        if is_correct:
            correct_predictions += 1
            status = "✅ CORRECT"
        else:
            status = "❌ INCORRECT"
        
        print(f"Expected: {expected}")
        print(f"Actual: {actual}")
        print(f"Result: {status}")
        
        results.append({
            "test_id": i,
            "message": message,
            "step": step,
            "expected": expected,
            "actual": actual,
            "correct": is_correct
        })
    
    # Summary
    accuracy = (correct_predictions / len(test_scenarios)) * 100
    
    print(f"\n{'='*60}")
    print("ROUTER TESTING SUMMARY")
    print(f"{'='*60}")
    print(f"Total Tests: {len(test_scenarios)}")
    print(f"Correct: {correct_predictions}")
    print(f"Incorrect: {len(test_scenarios) - correct_predictions}")
    print(f"Accuracy: {accuracy:.1f}%")
    
    # Show incorrect predictions
    incorrect_tests = [r for r in results if not r["correct"]]
    if incorrect_tests:
        print(f"\n❌ INCORRECT PREDICTIONS:")
        for test in incorrect_tests:
            print(f"  Test {test['test_id']}: '{test['message']}' in {test['step']}")
            print(f"    Expected: {test['expected']}, Got: {test['actual']}")
    
    return results

def test_step_completion_detection():
    """Test step completion detection logic."""
    
    print(f"\n{'='*60}")
    print("STEP COMPLETION DETECTION TESTS")
    print(f"{'='*60}")
    
    # Test scenarios for step completion
    completion_tests = [
        {
            "step": "name_verification",
            "state": {"name_verification_status": "VERIFIED"},
            "expected": True,
            "description": "Name verified"
        },
        {
            "step": "name_verification", 
            "state": {"name_verification_status": "INSUFFICIENT_INFO"},
            "expected": False,
            "description": "Name not verified"
        },
        {
            "step": "details_verification",
            "state": {"details_verification_status": "VERIFIED"},
            "expected": True,
            "description": "Details verified"
        },
        {
            "step": "reason_for_call",
            "state": {"messages": [
                {"type": "ai", "content": "Your account is overdue by R199.00"}
            ]},
            "expected": True,
            "description": "Reason explained"
        },
        {
            "step": "negotiation",
            "state": {"messages": [
                {"type": "ai", "content": "Services will stop without payment"},
                {"type": "human", "content": "Okay, I understand"}
            ]},
            "expected": True,
            "description": "Client shows agreement"
        },
        {
            "step": "promise_to_pay",
            "state": {"messages": [
                {"type": "human", "content": "Yes, debit my account"},
                {"type": "ai", "content": "Perfect! I'm setting up the payment now"}
            ]},
            "expected": True,
            "description": "Payment arrangement secured"
        }
    ]
    
    def is_step_complete(step: str, state: Dict[str, Any]) -> bool:
        """Mock step completion detection logic."""
        
        if step == "name_verification":
            return state.get("name_verification_status") == "VERIFIED"
            
        elif step == "details_verification":
            return state.get("details_verification_status") == "VERIFIED"
            
        elif step == "reason_for_call":
            messages = state.get("messages", [])
            for msg in messages:
                if msg.get("type") == "ai":
                    content = msg.get("content", "").lower()
                    if any(indicator in content for indicator in ["overdue", "payment", "owe", "balance"]):
                        return True
            return False
            
        elif step == "negotiation":
            messages = state.get("messages", [])
            for msg in reversed(messages):
                if msg.get("type") == "human":
                    content = msg.get("content", "").lower()
                    agreement_indicators = ["okay", "fine", "yes", "understand", "let's do it"]
                    if any(indicator in content for indicator in agreement_indicators):
                        return True
                    break
            return False
            
        elif step == "promise_to_pay":
            messages = state.get("messages", [])
            for msg in reversed(messages):
                if msg.get("type") == "ai":
                    content = msg.get("content", "").lower()
                    secured_indicators = ["perfect", "excellent", "great", "setting up", "arranging", "processing"]
                    if any(indicator in content for indicator in secured_indicators):
                        return True
                    break
            return False
            
        return False
    
    correct = 0
    for i, test in enumerate(completion_tests, 1):
        actual = is_step_complete(test["step"], test["state"])
        expected = test["expected"]
        is_correct = actual == expected
        
        if is_correct:
            correct += 1
            status = "✅"
        else:
            status = "❌"
        
        print(f"{status} Test {i}: {test['description']}")
        print(f"   Step: {test['step']}")
        print(f"   Expected: {expected}, Actual: {actual}")
    
    accuracy = (correct / len(completion_tests)) * 100
    print(f"\nStep Completion Accuracy: {accuracy:.1f}% ({correct}/{len(completion_tests)})")

def test_router_with_conversation_flow():
    """Test router with realistic conversation flow."""
    
    print(f"\n{'='*60}")
    print("CONVERSATION FLOW ROUTER TESTS")
    print(f"{'='*60}")
    
    # Simulate a conversation flow
    conversation = [
        {"step": "introduction", "ai": "Good day, may I speak to John Smith?", "human": "Speaking"},
        {"step": "name_verification", "ai": "Just to confirm, this is John Smith?", "human": "Yes, that's me"},
        {"step": "details_verification", "ai": "Please provide your ID number", "human": "8001010001088"},
        {"step": "reason_for_call", "ai": "Your account is overdue by R199", "human": "How does Cartrack work?"},
        {"step": "query_resolution", "ai": "Vehicle tracking. Now about that R199...", "human": "I can't afford it"},
        {"step": "negotiation", "ai": "Services stop without payment", "human": "Okay, let's arrange something"},
        {"step": "promise_to_pay", "ai": "Can we debit R199 today?", "human": "I want to cancel"},
    ]
    
    for i, turn in enumerate(conversation):
        print(f"\n💬 Conversation Turn {i+1}")
        print(f"Step: {turn['step']}")
        print(f"AI: {turn['ai']}")
        print(f"Human: {turn['human']}")
        
        classification = test_router_classification(
            turn["human"], 
            turn["step"], 
            turn["ai"]
        )
        
        # Predict what should happen next
        if classification == "QUERY_UNRELATED":
            print("🔄 Should route to: query_resolution")
        elif classification == "CANCELLATION":
            print("🔄 Should route to: cancellation")
        elif classification == "ESCALATION":
            print("🔄 Should route to: escalation")
        else:
            print(f"🔄 Should continue in: {turn['step']} or progress based on completion")

if __name__ == "__main__":
    print("🚀 Starting Router Tests...")
    
    # Run all tests
    run_comprehensive_router_tests()
    test_step_completion_detection()
    test_router_with_conversation_flow()
    
    print(f"\n✅ Router testing completed!")

2025-05-29 12:49:08,290 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🚀 Starting Router Tests...

COMPREHENSIVE ROUTER TESTING

📋 Test 1/22

🔍 TESTING ROUTER CLASSIFICATION
Current Step: negotiation
Test Message: 'I want to speak to a supervisor'
🎯 Router Classification: ESCALATION
🤖 Raw Response: ESCALATION
Expected: ESCALATION
Actual: ESCALATION
Result: ✅ CORRECT

📋 Test 2/22

🔍 TESTING ROUTER CLASSIFICATION
Current Step: promise_to_pay
Test Message: 'Get me your manager now'


2025-05-29 12:49:08,404 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-05-29 12:49:08,522 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🎯 Router Classification: ESCALATION
🤖 Raw Response: ESCALATION
Expected: ESCALATION
Actual: ESCALATION
Result: ✅ CORRECT

📋 Test 3/22

🔍 TESTING ROUTER CLASSIFICATION
Current Step: reason_for_call
Test Message: 'I'm filing a complaint'
🎯 Router Classification: ESCALATION
🤖 Raw Response: ESCALATION
Expected: ESCALATION
Actual: ESCALATION
Result: ✅ CORRECT

📋 Test 4/22

🔍 TESTING ROUTER CLASSIFICATION
Current Step: negotiation
Test Message: 'This is harassment, I want a supervisor'


2025-05-29 12:49:08,655 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-05-29 12:49:08,777 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

🎯 Router Classification: AGREEMENT
🤖 Raw Response: AGREEMENT
🔄 Should continue in: name_verification or progress based on completion

💬 Conversation Turn 3
Step: details_verification
AI: Please provide your ID number
Human: 8001010001088

🔍 TESTING ROUTER CLASSIFICATION
Current Step: details_verification
Test Message: '8001010001088'
AI Context: 'Please provide your ID number'
🎯 Router Classification: STEP_RELATED
🤖 Raw Response: STEP_RELATED
🔄 Should continue in: details_verification or progress based on completion

💬 Conversation Turn 4
Step: reason_for_call
AI: Your account is overdue by R199
Human: How does Cartrack work?

🔍 TESTING ROUTER CLASSIFICATION
Current Step: reason_for_call
Test Message: 'How does Cartrack work?'
AI Context: 'Your account is overdue by R199'


2025-05-29 12:49:11,258 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-05-29 12:49:11,378 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🎯 Router Classification: QUERY_UNRELATED
🤖 Raw Response: QUERY_UNRELATED
🔄 Should route to: query_resolution

💬 Conversation Turn 5
Step: query_resolution
AI: Vehicle tracking. Now about that R199...
Human: I can't afford it

🔍 TESTING ROUTER CLASSIFICATION
Current Step: query_resolution
Test Message: 'I can't afford it'
AI Context: 'Vehicle tracking. Now about that R199...'
🎯 Router Classification: OBJECTION
🤖 Raw Response: OBJECTION
🔄 Should continue in: query_resolution or progress based on completion

💬 Conversation Turn 6
Step: negotiation
AI: Services stop without payment
Human: Okay, let's arrange something

🔍 TESTING ROUTER CLASSIFICATION
Current Step: negotiation
Test Message: 'Okay, let's arrange something'
AI Context: 'Services stop without payment'


2025-05-29 12:49:11,494 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-05-29 12:49:11,610 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


🎯 Router Classification: AGREEMENT
🤖 Raw Response: AGREEMENT
🔄 Should continue in: negotiation or progress based on completion

💬 Conversation Turn 7
Step: promise_to_pay
AI: Can we debit R199 today?
Human: I want to cancel

🔍 TESTING ROUTER CLASSIFICATION
Current Step: promise_to_pay
Test Message: 'I want to cancel'
AI Context: 'Can we debit R199 today?'
🎯 Router Classification: CANCELLATION
🤖 Raw Response: CANCELLATION
🔄 Should route to: cancellation

✅ Router testing completed!
